# Setup Variables


In [1]:
aws_profile_name = "dev"
aws_region = "us-east-1"

In [2]:
import boto3

boto3.setup_default_session(profile_name=aws_profile_name)
BEDROCK_CLIENT = boto3.client("bedrock-runtime", aws_region)

# Example Without Tools


In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

chat_model = ChatBedrock(
    client=BEDROCK_CLIENT,
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs={"max_tokens": 1000, "temperature": 0.2, "top_p": 0.9},
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. If asked a question you do not know the answer to, simply respond with 'I don't know' instead of guessing.",
        ),
        ("human", "{question}"),
    ]
)
chain = prompt | chat_model

In [4]:
message = chain.invoke({"question": "What is the temperature in Urbandale, IA today?"})
print(message.content)

I don't know.


# Define Tools


In [5]:
def get_temp(city: str, state: str):
    """
    Would normally call out to an API or possibly another
    GenAI solution. However, in this case, just return
    that is is 72 degrees no matter where we are, so
    we can pretend we live in a wonderful climate.
    """
    print(f"get_temp tool called with city: {city} and state: {state}")
    return "72 Degrees Fahrenheit"


get_temp_description = """
<tool_description>
    <tool_name>get_temp</tool_name>
    <description>
        Returns temperature data for a given city and state.
    </description>
    <parameters>
        <parameter>
            <name>city</name>
            <type>string</type>
            <description>The city as a string</description>
        </parameter>
        <parameter>
            <name>state</name>
            <type>string</type>
            <description>The state as a string</description>
        </parameter>
    </parameters>
</tool_description>
""".strip()

all_tool_descriptions = "\n".join([get_temp_description])

## Helper functions when translating to and from model


In [6]:
def call_function(tool_name, parameters):
    func = globals()[tool_name]
    output = func(**parameters)
    return output


def format_result(tool_name, output):
    return f"""
<function_results>
<result>
<tool_name>{tool_name}</tool_name>
<stdout>
{output}
</stdout>
</result>
</function_results>
""".strip()

## Remake model with new stop sequence added

## Add tools definitions to system prompt


In [7]:
function_call_tag_start = "<function_calls>"
function_call_tag_end = "</function_calls>"

chat_model = ChatBedrock(
    client=BEDROCK_CLIENT,
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs={
        "max_tokens": 1000,
        "temperature": 0.2,
        "top_p": 0.9,
        "stop_sequences": ["Human: ", function_call_tag_end],
    }
)
messages = [
    (
        "system",
        """
            You are a helpful assistant. If asked a question you do not know the answer to, simply respond with 'I don't know' instead of guessing.
            
            In this environment you have access to a set of tools you can use to answer the user's question.

            You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
            <function_calls>
                <invoke>
                    <tool_name>$TOOL_NAME</tool_name>
                    <parameters>
                        <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
                        ...
                    </parameters>
                </invoke>
            </function_calls>

            Here are the tools available:
            <tools>
            {tools_string}
            </tools>
        """.strip(),
    ),
    ("human", "{question}"),
]
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | chat_model

## Invoke chain on a loop. If what it returns ends because of the new stop reason instead of the normal one, cut out the invoke portion, and use the info to call our function. Then add the AI message and a new user message with the tool call result and call again.


#### NOTE: For some reason, langchain does not return stop_reason, which I feel like it should. Instead, we just have to check if we ended in </invoke> to see if it ended in a function call


In [8]:
import xmltodict

while True:
    resp = chain.invoke(
        {
            "question": "What is the temperature in Urbandale, IA today?",
            "tools_string": all_tool_descriptions,
        }
    )
    message = str(resp.content)

    if function_call_tag_start in message:
        print("FUNCTION CALL MESSAGE:")
        print(message)
        # Find the function_calls xml portion of the response.
        start_index = message.find(function_call_tag_start) + len(
            function_call_tag_start
        )
        calls = message[start_index:-1]

        # Turn that XML into dictionaries
        dict_calls = xmltodict.parse(calls)

        # Call the appropriate tools
        call = dict_calls["invoke"]
        answer = call_function(**call)

        # Format the result of the tool call into XML so the model
        # can parse it better
        formatted_answer = format_result(call["tool_name"], answer)
        print("TOOL RESPONSE:")
        print(formatted_answer)

        # "Continue" the conversation with the model from where we
        # left off by updating our messages to include the tool response
        prompt = ChatPromptTemplate.from_messages(
            messages + [resp, ("human", formatted_answer)]
        )
        chain = prompt | chat_model
    else:
        print("FINAL ANSWER:")
        print(resp.content)
        break

FUNCTION CALL MESSAGE:
To answer your question about the temperature in Urbandale, IA today, I'll need to use the get_temp tool to retrieve that information. Let me do that for you.

<function_calls>
    <invoke>
        <tool_name>get_temp</tool_name>
        <parameters>
            <city>Urbandale</city>
            <state>IA</state>
        </parameters>
    </invoke>

get_temp tool called with city: Urbandale and state: IA
TOOL RESPONSE:

<function_results>
<result>
<tool_name>get_temp</tool_name>
<stdout>
72 Degrees Fahrenheit
</stdout>
</result>
</function_results>

FINAL ANSWER:
Based on the information I've retrieved, the temperature in Urbandale, IA today is 72 degrees Fahrenheit.

This is a comfortable temperature, typical of a pleasant day. It's neither too hot nor too cold, making it suitable for various outdoor activities. Remember that temperatures can change throughout the day, so this might be the current temperature or an average for the day.

Is there anything else y